In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#EDA

In [2]:
df_products = pd.read_csv(r'C:\Users\shanm\Downloads\product_info.csv\product_info.csv')

In [3]:
df_products.shape
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

In [4]:
df_products.isna().sum(axis=0)

product_id               0
product_name             0
brand_id                 0
brand_name               0
loves_count              0
rating                 278
reviews                278
size                  1631
variation_type        1444
variation_value       1598
variation_desc        7244
ingredients            945
price_usd                0
value_price_usd       8043
sale_price_usd        8224
limited_edition          0
new                      0
online_only              0
out_of_stock             0
sephora_exclusive        0
highlights            2207
primary_category         0
secondary_category       8
tertiary_category      990
child_count              0
child_max_price       5740
child_min_price       5740
dtype: int64

In [5]:
df_products.groupby(['brand_name'])['product_name'].count().sort_values(ascending=False).head(10)

brand_name
SEPHORA COLLECTION         352
CLINIQUE                   179
Dior                       136
tarte                      131
NEST New York              115
Bumble and bumble          110
Kérastase                  108
TOM FORD                   100
Charlotte Tilbury           99
Anastasia Beverly Hills     95
Name: product_name, dtype: int64

In [6]:
#popular product_category in each brand

tmp = pd.DataFrame(df_products.groupby(['brand_name','primary_category'])['product_id'].count().reset_index())
tmp['product_rank'] = tmp.groupby(['brand_name'])['product_id'].rank(method = 'min',ascending = False)
tmp[tmp.product_rank == 1]

,brand_name,primary_category,product_id,product_rank
0,19-69,Fragrance,14,1.0
1,54 Thrones,Bath & Body,4,1.0
2,ABBOTT,Fragrance,13,1.0
3,AERIN,Fragrance,22,1.0
6,ALTERNA Haircare,Hair,45,1.0
...,...,...,...,...
618,philosophy,Bath & Body,1,1.0
619,rms beauty,Makeup,11,1.0
620,shu uemura,Hair,43,1.0
621,stila,Makeup,10,1.0


In [7]:
#popular product in each brand
tmp1 = df_products[['brand_name','product_name','loves_count']]
tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)
tmp1

C:\Users\shanm\AppData\Local\Temp/ipykernel_28644/3412678004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)


,brand_name,product_name,loves_count,rank
0,19-69,Fragrance Discovery Set,6320,1.0
1,19-69,La Habana Eau de Parfum,3827,2.0
2,19-69,Rainbow Bar Eau de Parfum,3253,3.0
3,19-69,Kasbah Eau de Parfum,3018,4.0
4,19-69,Purple Haze Eau de Parfum,2691,5.0
...,...,...,...,...
8489,Yves Saint Laurent,Couture Clutch Eyeshadow Palette,2790,61.0
8490,Yves Saint Laurent,L'Homme Eau de Parfum,2319,62.0
8491,Yves Saint Laurent,Mon Paris Eau de Parfum Gift Set,1475,63.0
8492,Yves Saint Laurent,Y Eau de Parfum Gift Set,840,64.0


In [8]:
#Average price of each product category
tmp = df_products[['primary_category','secondary_category','tertiary_category','price_usd']]
tmp1 = pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].min().reset_index())
tmp1['max_price']=pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].max().reset_index())['price_usd']
tmp1['mean_price']=pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].mean().reset_index())['price_usd']
tmp1.rename(columns={'price_usd':'min_price'})

,primary_category,secondary_category,tertiary_category,min_price,max_price,mean_price
0,Bath & Body,Bath & Shower,Bath Soaks & Bubble Bath,3.0,82.0,29.142857
1,Bath & Body,Bath & Shower,Body Wash & Shower Gel,6.0,70.0,30.705882
2,Bath & Body,Bath & Shower,Scrub & Exfoliants,18.0,155.0,43.692308
3,Bath & Body,Body Care,Cellulite & Stretch Marks,40.0,77.0,62.625000
4,Bath & Body,Body Care,Deodorant & Antiperspirant,8.0,96.0,28.920000
...,...,...,...,...,...,...
130,Tools & Brushes,Brushes & Applicators,Facial Cleansing Brushes,14.0,16.0,15.000000
131,Tools & Brushes,Brushes & Applicators,Sponges & Applicators,10.0,14.0,12.000000
132,Tools & Brushes,Hair Tools,Accessories,15.0,45.0,30.000000
133,Tools & Brushes,Hair Tools,Brushes & Combs,7.0,74.0,26.000000


In [9]:
df_products.ingredients.str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.split(',',expand=True)

,0,1,2,3,4,5,6,7,8,9,...,710,711,712,713,714,715,716,717,718,719
0,'Capri Eau de Parfum:','Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) D-Limonene,Linalool,Benzyl Salicylate,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Benzl Benzoate,Citral,...,None,None,None,None,None,None,None,None,None,None
1,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Ethylhexyl Methoxycinnamate,Ethylhexyl Salicylate,Butyl Methoxydibenzoylmethane,Benzyl Alcohol,Benzyl Benzoate,Benzyl Cinnamate,Cinnamal,Citral,Coumarin Limonene,...,None,None,None,None,None,None,None,None,None,None
2,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) D-Limonene,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Hexyl Cinnamal Citronellol,Linalool,Coumarin,Eugenol,Citral,...,None,None,None,None,None,None,None,None,None,None
3,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Coumarin,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,D-Limonene,Eugenol,Linalool,Citronellol,Geraniol,...,None,None,None,None,None,None,None,None,None,None
4,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Octoxynol-10,D-Limonene,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Geraniol,Linalool,Coumarin,Farnesol,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,'Talc,Synthetic Fluorphlogopite,Triethylhexanoin,Octyldodecyl Stearoyl Stearate,Ci 77891 / Titanium Dioxide,Caprylyl Glycol,Ci 77491,Ci 77499 / Iron Oxides,Dehydroacetic Acid,Phenoxyethanol,...,None,None,None,None,None,None,None,None,None,None
8490,'Alcohol,Aqua / Water / Eau,Parfum / Fragrance,Limonene,Linalool,Coumarin,Butyl Methoxydibenzoylmethane,Alpha-Isomethyl Ionone,Geraniol,Citral,...,None,None,None,None,None,None,None,None,None,None
8491,'Mon Paris Eau de Parfum:','Alcohol,Parfum/Fragrance,Aqua/Water,Benzyl Salicylate,Linalool,Benzyl Alcohol,Hydroxycitronellal,Ethylhexyl Methoxycinnamate,Limonene,...,None,None,None,None,None,None,None,None,None,None
8492,'Alcohol,Parfum/Fragrance,Aqua/Water,Limonene,Linalool,Butyl Methoxydibenzoylmethane,Citral,Citronellol,Alpha-Isomethyl Ionone,Geraniol,...,None,None,None,None,None,None,None,None,None,None


#Sentiment Analysis

In [10]:
reviews_250 = pd.read_csv(r"C:\Users\shanm\Downloads\reviews_0_250.csv\reviews_0_250.csv",low_memory = False)
# reviews_500 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_250_500.csv',low_memory = False)
# reviwes_750 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_500_750.csv',low_memory = False)
# reviwes_1000 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_750_1000.csv',low_memory = False)
# reviwes_1500 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_1000_1500.csv',low_memory = False)


In [11]:
reviews = pd.concat([reviews_250])##, reviews_500, reviwes_750, reviwes_1000 , reviwes_1500])
reviews = reviews[['review_text' , 'is_recommended' , 'rating']]
reviews

,review_text,is_recommended,rating
0,I use this with the Nudestix “Citrus Clean Bal...,1.0,5
1,I bought this lip mask after reading the revie...,0.0,1
2,My review title says it all! I get so excited ...,1.0,5
3,I’ve always loved this formula for a long time...,1.0,5
4,"If you have dry cracked lips, this is a must h...",1.0,5
...,...,...,...
602125,I can’t say enough great things about this pro...,1.0,5
602126,I have been using this product for over 6 week...,1.0,5
602127,I love the Triple Algae Eye Renewal Balm! Once...,1.0,5
602128,This made the lines around my eyes so much bet...,1.0,5


In [12]:
reviews.rename(columns = {'is_recommended':'label'},inplace=True)
reviews.isna().sum()

review_text       999
label          117486
rating              0
dtype: int64

In [13]:
reviews.isnull().sum()
reviews.dropna(inplace=True)

In [14]:
reviews.label.astype(int)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483645 entries, 0 to 602129
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_text  483645 non-null  object 
 1   label        483645 non-null  float64
 2   rating       483645 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.8+ MB


In [15]:
num_neg = pd.DataFrame(reviews.label.value_counts().reset_index()).iloc[1,1]
num_neg

78410

In [16]:
reviews_sample = reviews.groupby('label').apply(lambda x : x.sample(num_neg))
reviews_sample.label.value_counts()

0.0    78410
1.0    78410
Name: label, dtype: int64

#text pre-processing

In [17]:
from sklearn.utils import shuffle
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
reviews_sample= shuffle(reviews_sample)
reviews_sample.reset_index(drop=True, inplace= True)
reviews_sample

,review_text,label,rating
0,Gel type moisturizers aren’t typically my go-t...,1.0,5
1,smells great! works really well on my acne sca...,1.0,4
2,Caused me so many breakouts. I thought this wo...,0.0,1
3,I could not be happier with this mask! I alway...,1.0,5
4,Fastest acting spot reducer I have ever tried....,1.0,5
...,...,...,...
156815,I got this complimentary in exchange for revie...,0.0,2
156816,I have been taking care of my skin religiously...,0.0,1
156817,I love love love this product!!! Works amazing...,1.0,5
156818,I purchased the Sephora Collection Ultra Glow ...,0.0,2


In [19]:
#reviews_sample.to_csv('/kaggle/working/reviews_sample.csv')

In [20]:
#reviews_sample = pd.read_csv('/kaggle/working/reviews_sample.csv')

In [19]:
#number of words in each review
reviews_sample['text_word_count']=reviews_sample['review_text'].apply(lambda x:len(x.split()))
reviews_sample['text_word_count']

0          51
1          44
2          29
3          45
4          50
         ... 
156815     55
156816     82
156817     29
156818    113
156819    246
Name: text_word_count, Length: 156820, dtype: int64

To prepare the text data for the model building we perform text preprocessing. It is the very first step of NLP projects. Some of the preprocessing steps are:

Removing punctuations \
Removing URLs\
Removing Stop words\
Lower casing\
Tokenization\
Stemming\
Lemmatization

In [20]:
# removing punctuations
def remove_punc(text):
    
    clean_text = "".join([i for i in text if i not in string.punctuation])
    return clean_text

reviews_sample['clean_text']  = reviews_sample['review_text'].apply(lambda x: remove_punc(x))


In [21]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [22]:
# removing HTML text
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
reviews_sample['clean_text']  = reviews_sample['clean_text'].apply(lambda x: strip_html(x))


In [23]:
# converting to lower case
reviews_sample['clean_text']  = reviews_sample['clean_text'].apply(lambda x: x.lower())

In [24]:
import re
def remove_special_characters(text, remove_digits=True):
    text = re.sub("\d", "", text)
    return text
reviews_sample['clean_text']  = reviews_sample['clean_text'].apply(remove_special_characters)

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shanm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
#stop words removal
import nltk 
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize 
stopwords[0:10]
def remove_stopwords(text):
    text = [i for i in word_tokenize(text) if i not in stopwords]
    return text
reviews_sample['tokenized_text']  = reviews_sample['clean_text'].apply(remove_stopwords)

In [27]:
#stemming
from nltk.stem.porter import PorterStemmer
porterstemmer = PorterStemmer()
def stemming(text):
    text =[porterstemmer.stem(word) for word in text]
    return text
reviews_sample['tokenized_text']  = reviews_sample['tokenized_text'].apply(stemming)

In [29]:
reviews_sample[['tokenized_text','clean_text']].iloc[0,:]

tokenized_text    [gel, type, moistur, ’, typic, goto, one, impr...
clean_text        gel type moisturizers aren’t typically my goto...
Name: 0, dtype: object

In [30]:
reviews_sample['tokenized_text_2']  = reviews_sample['tokenized_text'].apply(lambda x :' '.join(i for i in x))

In [31]:
reviews_sample[['tokenized_text_2','clean_text']].iloc[0,:]

tokenized_text_2    gel type moistur ’ typic goto one impress gel ...
clean_text          gel type moisturizers aren’t typically my goto...
Name: 0, dtype: object

In [32]:
X= reviews_sample['tokenized_text_2']
y = reviews_sample['label'] 

In [35]:
reviews_sample.to_csv(r'C:\Users\shanm\OneDrive\Desktop\Git_data\Sephora_sample\X_reviews_sample.csv')

In [106]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)

In [107]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # encoding
y_train_label= le.fit_transform(y_train)
y_test_label= le.transform(y_test)

In [108]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=20000)
vec.fit(X_train)
x_train=vec.transform(X_train)
x_test=vec.transform(X_test)

In [109]:
vec.transform(['happi hear work other hand oili complexion']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [110]:
vec.get_feature_names_out()[999]

'anyt'

In [112]:
find =vec.get_feature_names_out()
np.where(find=='favour')

(array([6702], dtype=int64),)

In [40]:
 x_train = x_train.toarray()

In [41]:
x_train.shape, y_train.shape

((78410, 20000), (78410,))

In [43]:
import tensorflow as tf

In [44]:
#y_train = tf.keras.utils.to_categorical(y_train)
#y_test = tf.keras.utils.to_categorical(y_test)

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(16, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                320016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,305
Trainable params: 320,305
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(x_train,y_train_label,epochs=10)

Epoch 1/10
2451/2451 [==============================] - 20s 8ms/step - loss: 0.0306 - accuracy: 0.9898
Epoch 2/10
2451/2451 [==============================] - 14s 6ms/step - loss: 0.0255 - accuracy: 0.9913
Epoch 3/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0211 - accuracy: 0.9929
Epoch 4/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0171 - accuracy: 0.9946
Epoch 5/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0154 - accuracy: 0.9952
Epoch 6/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0127 - accuracy: 0.9962
Epoch 7/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0114 - accuracy: 0.9963
Epoch 8/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0093 - accuracy: 0.9972
Epoch 9/10
2451/2451 [==============================] - 13s 5ms/step - loss: 0.0096 - accuracy: 0.9968
Epoch 10/10
2451/2451 [==============================] - 13s 5ms/step - l

In [70]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_val, X_test2, y_val, y_test2 = train_test_split(X, y ,test_size=0.80, shuffle = True)

In [71]:
y_val

145853    1.0
80141     0.0
4900      0.0
112454    0.0
82398     0.0
         ... 
98589     0.0
107640    0.0
64312     0.0
121527    0.0
146774    0.0
Name: label, Length: 31364, dtype: float64

In [72]:
x_val=vec.transform(X_val).toarray()
y_val =le.transform(y_val)

In [77]:
x_val.shape

(31364, 20000)

In [78]:
y_pred = model.predict(x_val)

In [96]:

condlist = [(y_pred>=0.5), (y_pred<0.5)]
choicelist = [1, 0]

y_classes = np.select(condlist, choicelist, default=y_pred)

In [98]:
y_classes

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [99]:
from sklearn.metrics import *
confusion_matrix(y_val,y_classes)

array([[14734,   914],
       [ 1066, 14650]], dtype=int64)

In [100]:
print(classification_report(y_val,y_classes))

'              precision    recall  f1-score   support\n\n           0       0.93      0.94      0.94     15648\n           1       0.94      0.93      0.94     15716\n\n    accuracy                           0.94     31364\n   macro avg       0.94      0.94      0.94     31364\nweighted avg       0.94      0.94      0.94     31364\n'

In [ ]:
Done